conda env automl

# autogluon.tabular

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
train = pd.read_csv('../data/train.csv', sep=",")
test = pd.read_csv('../data/test.csv')

import sklearn
y = train['SalePrice']
X = train.drop('SalePrice', axis=1)
X=X[set(X.columns.values.tolist())]
categorical = [var for var in X.columns if X[var].dtype=='O']
numerical = [var for var in X.columns if X[var].dtype!='O']
X[categorical] = X[categorical].fillna('None')

# auto-sklearn can not deal with categorical variables
X= pd.concat([pd.get_dummies(X[categorical], dummy_na=True), X[numerical]], axis=1)

y = np.log1p(y)
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, random_state=42, test_size=0.2)

/tmp/ipykernel_23622/3159152723.py:11: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  X=X[set(X.columns.values.tolist())]


In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor
X_train = pd.concat([X_train, y_train], axis=1)
X_test = pd.concat([X_test, y_test], axis=1)
train_data = TabularDataset(X_train)
test_data = TabularDataset(X_test)
predictor = TabularPredictor(label='SalePrice',eval_metric = 'root_mean_squared_error').fit(train_data, time_limit=5000,
                                                   presets='best_quality',auto_stack=True,)  # For a fair comparison with auto-sklearn
# leaderboard = predictor.leaderboard(test_data)


/home/martin/miniconda3/envs/automl/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No path specified. Models will be saved in: "AutogluonModels/ag-20240406_120855/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 5000s
AutoGluon will save models to "AutogluonModels/ag-20240406_120855/"
AutoGluon Version:  0.7.0
Python Version:     3.9.16
Operating System:   Linux
Platform Machine:   aarch64
Platform Version:   #1 SMP PREEMPT Thu May 25 07:27:39 UTC 2023
Train Data Rows:    1168
Train Data Columns: 347
Label Column: SalePrice
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observe

		`import lightgbm` failed. A quick tip is to install via `pip install autogluon.tabular[lightgbm]==0.7.0`.
Repeating k-fold bagging: 2/20
Repeating k-fold bagging: 3/20
Repeating k-fold bagging: 4/20
Repeating k-fold bagging: 5/20
Repeating k-fold bagging: 6/20
Repeating k-fold bagging: 7/20
Repeating k-fold bagging: 8/20
Repeating k-fold bagging: 9/20
Repeating k-fold bagging: 10/20
Repeating k-fold bagging: 11/20
Repeating k-fold bagging: 12/20
Repeating k-fold bagging: 13/20
Repeating k-fold bagging: 14/20
Repeating k-fold bagging: 15/20
Repeating k-fold bagging: 16/20
Repeating k-fold bagging: 17/20
Repeating k-fold bagging: 18/20
Repeating k-fold bagging: 19/20
Repeating k-fold bagging: 20/20
Completed 20/20 k-fold bagging repeats ...
Fitting model: WeightedEnsemble_L2 ... Training model for up to 360.0s of the 4997.0s of remaining time.
	-0.1416	 = Validation score   (-root_mean_squared_error)
	0.04s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting 9 L2 models ...
Fitti

In [3]:
y_test = test_data['SalePrice']  # values to predict
test_data_nolab = test_data.drop(columns=['SalePrice'])  # delete label column to prove we're not cheating
y_pred = predictor.predict(test_data_nolab)

perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: root_mean_squared_error on test data: -0.13554196496527673
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -0.13554196496527673,
    "mean_squared_error": -0.018371624266648307,
    "mean_absolute_error": -0.0895835047512191,
    "r2": 0.9015512194348443,
    "pearsonr": 0.9508906461329354,
    "median_absolute_error": -0.05542907908809358
}


In [4]:
results = predictor.fit_summary(show_plot=True)

*** Summary of fit() ***
Estimated performance of each model:
                    model  score_val  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0     WeightedEnsemble_L3  -0.136934       0.399790  3.075252                0.000137           0.022314            3       True          8
1    ExtraTreesMSE_BAG_L2  -0.137130       0.326495  2.019529                0.071598           0.610772            2       True          7
2  RandomForestMSE_BAG_L2  -0.140685       0.328055  2.442166                0.073159           1.033409            2       True          6
3     WeightedEnsemble_L2  -0.142021       0.147970  1.441790                0.000138           0.036772            2       True          5
4  RandomForestMSE_BAG_L1  -0.146030       0.071166  0.835099                0.071166           0.835099            1       True          3
5    ExtraTreesMSE_BAG_L1  -0.146088       0.073959  0.565807                0.073959           0.

/home/martin/miniconda3/envs/automl/lib/python3.9/site-packages/autogluon/core/utils/plots.py:138: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')
